<a href="https://colab.research.google.com/github/RajeshMallik19/Project_Q/blob/main/3_Employee_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Load the prepped data
df = pd.read_csv('/content/employee_attrition_prepped.csv')

# Separate features (X) and target (y)
X = df.drop(columns=['Attrition'])
y = df['Attrition']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Data loaded and prepped for Deep Learning.")

Data loaded and prepped for Deep Learning.


In [17]:
# Convert dataframes to numpy arrays with float dtype before reshaping
X_train_resampled_np = X_train_resampled.values.astype(float)
X_test_np = X_test.values.astype(float)

# Reshape the data to be 3D
# We treat our tabular data as a sequence with a single timestep.
X_train_reshaped = np.reshape(X_train_resampled_np, (X_train_resampled_np.shape[0], 1, X_train_resampled_np.shape[1]))
X_test_reshaped = np.reshape(X_test_np, (X_test_np.shape[0], 1, X_test_np.shape[1]))


# --- Verification Step ---
print(f"Original training data shape: {X_train_resampled_np.shape}")
print(f"Reshaped training data shape: {X_train_reshaped.shape}")
print(f"Original testing data shape: {X_test_np.shape}")
print(f"Reshaped testing data shape: {X_test_reshaped.shape}")

Original training data shape: (1972, 51)
Reshaped training data shape: (1972, 1, 51)
Original testing data shape: (294, 51)
Reshaped testing data shape: (294, 1, 51)


In [18]:
# --- Define the Model Architecture ---
model = Sequential([
    # Input shape is (timesteps, features)
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    Conv1D(filters=32, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5), # Helps prevent overfitting
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

# --- Compile the Model ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model
model.summary()

# --- Train the Model ---
print("\nTraining the BiLSTM-CNN model...")
history = model.fit(
    X_train_reshaped,
    y_train_resampled,
    epochs=20,          # We'll train for 20 passes over the data
    batch_size=32,
    validation_split=0.1, # Use 10% of training data for validation
    verbose=1           # Show the progress
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_5 (Bidirectional) │ (None, 1, 128)         │        59,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 1, 32)          │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,609 (252.38 KB)

 Trainable params: 64,609 (252.38 KB)

 Non-trainable params: 0 (0.00 B)


Training the BiLSTM-CNN model...
Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5186 - loss: 0.6981 - val_accuracy: 0.3636 - val_loss: 0.7718
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5909 - loss: 0.6597 - val_accuracy: 0.3687 - val_loss: 0.7599
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6361 - loss: 0.6373 - val_accuracy: 0.4949 - val_loss: 0.7268
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6205 - loss: 0.6433 - val_accuracy: 0.4444 - val_loss: 0.7616
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6064 - loss: 0.6580 - val_accuracy: 0.4646 - val_loss: 0.7140
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6197 - loss: 0.6498 - val_accuracy: 0.3485 - val_loss: 0.7574
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5937 - loss: 0.6554 - val_accuracy: 0.5505 - val_loss: 0.6671
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5638 - loss: 0.6624

In [19]:
from sklearn.metrics import classification_report

# Make predictions on the test set
# The model outputs probabilities, so we round to get 0 or 1
y_pred_dl_proba = model.predict(X_test_reshaped)
y_pred_dl = (y_pred_dl_proba > 0.5).astype(int)

# --- Evaluation Step ---
print("\n--- BiLSTM-CNN Model Evaluation ---")
print(classification_report(y_test, y_pred_dl))

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step

--- BiLSTM-CNN Model Evaluation ---
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       247
           1       0.21      0.23      0.22        47

    accuracy                           0.74       294
   macro avg       0.53      0.53      0.53       294
weighted avg       0.75      0.74      0.74       294

